In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

data_dir = './data/'
file = 'userid-timestamp-artid-artname-traid-traname.tsv'

In [2]:
df = pd.read_csv(data_dir + file, delimiter="\t", header=None, error_bad_lines=False)
df.describe()
drop_na_df = df.dropna(axis=0, how='any')
#drop_na_df.head(5)
drop_na_df.describe()

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


,0,1,2,3,4,5
count,16936134,16936134,16936134,16936134,16936134,16936134
unique,992,15631466,83905,81751,960402,693231
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,172042,193,111488,111488,3991,14908


In [3]:
file_name = 'original_set'
#drop_na_df.to_pickle(data_dir + file_name)
drop_na_df = pd.read_pickle(data_dir + file_name)

In [4]:
most_close_time = datetime.strptime("2009-05-05 23:59:59", "%Y-%m-%d %H:%M:%S")
print(most_close_time)
most_early_time = most_close_time - timedelta(days=180)
print(most_early_time)

2009-05-05 23:59:59
2008-11-06 23:59:59


In [5]:
temp_df = drop_na_df.copy()
f = lambda x: np.NaN if datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ") < most_early_time else x
temp_df.iloc[:,1:2] = drop_na_df.iloc[:,1:2].applymap(f)

split_date_df = temp_df.dropna(axis=0, how='any')
split_date_df.describe()

,0,1,2,3,4,5
count,3052646,3052646,3052646,3052646,3052646,3052646
unique,923,2757174,49603,48747,465798,353529
top,user_000791,2009-02-26T21:29:15Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,153d8ca0-dc23-4548-b64f-2c49db7f30db,Heartless
freq,48822,193,31412,31412,2559,2635


In [6]:
file_name = 'split_date_set'
split_date_df.to_pickle(data_dir + file_name)

In [7]:
Session_time = timedelta(minutes=20)

In [8]:
dataset_dic = {}
#for idx in range(split_date_df.shape[0]-1,-1,-1):
for idx in range(split_date_df.shape[0]-1,-1,-1):
    data = split_date_df.iloc[idx:idx+1,:].values[0]
    #print(data)
    if not data[0][5:] in dataset_dic.keys():
        #Session List
        dataset_dic[data[0][5:]] = []
        current_time = datetime.strptime(data[1], "%Y-%m-%dT%H:%M:%SZ")
        #Session
        dataset_dic[data[0][5:]].append([])
        #log
        dataset_dic[data[0][5:]][0].append([data[3],data[5]])
    else:
        #log time
        temp_time = datetime.strptime(data[1], "%Y-%m-%dT%H:%M:%SZ")
        if temp_time - current_time > Session_time:
            #new session
            dataset_dic[data[0][5:]].append([])
            #Put the log to the len(session_list)-1 of session_list(current_session)
            dataset_dic[data[0][5:]][len(dataset_dic[data[0][5:]])-1].append([data[3],data[5]])
        else:
            #Put the log to the len(session_list)-1 of session_list(current_session)
            dataset_dic[data[0][5:]][len(dataset_dic[data[0][5:]])-1].append([data[3],data[5]])

In [9]:
dataset_dic.keys()

dict_keys(['001000', '000999', '000998', '000997', '000995', '000994', '000993', '000992', '000991', '000990', '000989', '000987', '000986', '000985', '000984', '000983', '000982', '000981', '000980', '000979', '000978', '000977', '000976', '000975', '000974', '000973', '000972', '000971', '000970', '000969', '000968', '000967', '000966', '000965', '000964', '000963', '000962', '000961', '000960', '000959', '000958', '000957', '000956', '000955', '000954', '000953', '000952', '000951', '000950', '000949', '000948', '000947', '000946', '000945', '000944', '000943', '000942', '000941', '000940', '000939', '000938', '000937', '000936', '000935', '000934', '000933', '000932', '000931', '000930', '000929', '000928', '000927', '000926', '000925', '000924', '000923', '000922', '000921', '000920', '000919', '000918', '000917', '000916', '000915', '000914', '000913', '000912', '000911', '000910', '000909', '000908', '000907', '000906', '000905', '000904', '000903', '000901', '000900', '000899',

In [10]:
a = list(dataset_dic.values())

In [11]:
print(len(a))
print(len(a[1]))
for i in range(7357):
    print(a[1][i])
    break
print(len(a[1][0]))

923
13324
[['Billy Talent', 'Pins And Needles'], ['Billy Talent', 'Surrender'], ['Billy Talent', 'Devil In A Midnight Mass'], ['Billy Talent', 'Fallen Leaves'], ['Billy Talent', 'Pins And Needles'], ['Billy Talent', 'Red Flag'], ['Billy Talent', 'Surrender']]
7
